In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import hamming_loss,accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

import os

from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv',header=None)
train_target_df = pd.read_csv('trainLabels.csv')
submission_df = pd.read_csv('sampleSubmission_small.csv')

In [ ]:
print("train_df --> ",train_df.shape)
print("test_df --> ",test_df.shape)
print("train_target_df --> ",train_target_df.shape)
print("submission_df --> ",submission_df.shape)

train_df -->  (4698, 146)
test_df -->  (2000, 146)
train_target_df -->  (49999, 34)
submission_df -->  (990, 2)


In [ ]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
0,1698001,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.414798,0.000000,1.000000,0.000000,0.202060,...,0.0,1.000000,1262,892,NO,NO,NO,0,0.089686,0.193344
1,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,...,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
2,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,...,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
3,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.870000,4672,3306,YES,NO,YES,0,0.870538,0.405822
4,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,...,0.0,0.870000,4400,3413,YES,NO,YES,2,0.224729,0.870909


In [ ]:
train_target_df.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,...,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
0,1698001,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.414798,0.000000,1.000000,0.000000,0.202060,...,0.0,1.000000,1262,892,NO,NO,NO,0,0.089686,0.193344
1,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,...,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
2,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,...,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
3,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.870000,4672,3306,YES,NO,YES,0,0.870538,0.405822
4,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,...,0.0,0.870000,4400,3413,YES,NO,YES,2,0.224729,0.870909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1699996,NO,NO,Z6vucL/W0MPoFsgu2ewNXrvNCAQFiKzUJTYuqh6lP28=,yhI9Bw5Q8l1vEll4sw/Tem/jojpE9KwjKvQQIyrAqgU=,1.294118,0.000000,1.000000,0.000000,0.164141,...,0.0,1.000000,1188,918,YES,NO,YES,2,0.198257,0.155724
1996,1699997,NO,NO,LKQ9Uh6tQ3ZrIxAKaPaDEuiYFunnK/2d+oKAfpN9tuY=,h0cPLYjd7nmw9FJsQA+KUsnChH0SajbHjNdfMk47k9o=,1.020217,0.583944,0.625842,1.003516,0.791136,...,0.0,0.720000,4400,3413,YES,NO,YES,0,0.582479,0.778864
1997,1699998,NO,NO,/tuZYGMsFx4A/Ou+jSol6t/TpLRkSl8Ku+1tnQPvwww=,aLEeZ8ZFKt2jQfkG5e9Nmad+QJlfpPmSfQS3CHlL6Ik=,0.354706,0.550882,0.930882,0.207941,0.207500,...,0.0,0.845000,4400,3400,NO,NO,NO,5,0.930588,0.201591
1998,1699999,NO,NO,uMIU2KDOxlgzhYToCFCa3nMxIOPV0WqCnKWfooGaw+8=,4LhhvTzxwvh2SnFtcpaRasyvph66a3YDIQCshAfyS2o=,1.220588,0.102059,0.326176,1.213824,0.942955,...,1.5,0.676667,4400,3400,NO,NO,NO,2,0.256471,0.938182


In [ ]:
test_df.columns=['id',
                 'x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',
                 'x11','x12','x13','x14','x15','x16','x17','x18','x19','x20',
                 'x21','x22','x23','x24','x25','x26','x27','x28','x29','x30',
                 'x31','x32','x33','x34','x35','x36','x37','x38','x39','x40',
                 'x41','x42','x43','x44','x45','x46','x47','x48','x49','x50',
                 'x51','x52','x53','x54','x55','x56','x57','x58','x59','x60',
                 'x61','x62','x63','x64','x65','x66','x67','x68','x69','x70',
                 'x71','x72','x73','x74','x75','x76','x77','x78','x79','x80',
                 'x81','x82','x83','x84','x85','x86','x87','x88','x89','x90',
                 'x91','x92','x93','x94','x95','x96','x97','x98','x99','x100',
                 'x101','x102','x103','x104','x105','x106','x107','x108','x109','x110',
                 'x111','x112','x113','x114','x115','x116','x117','x118','x119','x120',
                 'x121','x122','x123','x124','x125','x126','x127','x128','x129','x130',
                 'x131','x132','x133','x134','x135','x136','x137','x138','x139','x140',
                 'x141','x142','x143','x144','x145']

In [ ]:
test_df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1698001,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.414798,0.000000,1.000000,0.000000,0.202060,...,0.0,1.000000,1262,892,NO,NO,NO,0,0.089686,0.193344
1,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,...,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
2,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,...,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
3,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.870000,4672,3306,YES,NO,YES,0,0.870538,0.405822
4,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,...,0.0,0.870000,4400,3413,YES,NO,YES,2,0.224729,0.870909


In [ ]:
train_df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,...,0.0,0.810,3306.0,4676.0,YES,NO,YES,2.0,0.375535,0.464610
1,2,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.510,4678.0,3306.0,YES,NO,YES,4.0,0.741682,0.593630
2,3,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,...,0.0,0.850,4678.0,3306.0,NO,NO,NO,1.0,0.776467,0.493159
3,4,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.041471,0.940787,0.090851,0.556564,...,0.0,0.945,3306.0,4678.0,NO,NO,YES,3.0,0.168234,0.546582
4,5,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.415919,0.000000,1.000000,0.000000,0.375297,...,0.0,1.000,1263.0,892.0,NO,NO,NO,1.0,0.246637,0.361045


In [ ]:
#All rows that has atleast one NAN value
train_df[train_df.isna().any(axis=1)]

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
1,2,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.51,4678.0,3306.0,YES,NO,YES,4.0,0.741682,0.593630
8,9,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.82,4675.0,3304.0,YES,NO,YES,1.0,0.923426,0.702032
12,13,NO,NO,vIR+5O6D5pKgFw/CFbDtHBZ/AhaNMnxLM4BTIwXUres=,gNVKedeHcKHyWnIMnBCE9t9XOschRQCU8/XfcaY0mMk=,0.440411,0.083787,0.776165,0.253781,0.210774,...,0.0,0.74,4678.0,3306.0,YES,NO,YES,0.0,0.029643,0.199230
25,26,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.00,1188.0,918.0,YES,NO,YES,5.0,0.898693,0.502525
26,27,NO,NO,EKynK11V/TnbPzoD8gy12kkm85wTNzN4kciauQQOHO8=,LXzIvxHIblNu5ThzWnaNlPxZrF3BRg4q4JWZPXQ5Z40=,1.415919,0.000000,1.000000,0.000000,0.889153,...,0.0,1.00,1263.0,892.0,YES,NO,YES,0.0,0.032511,0.883610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4684,4685,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.00,1263.0,893.0,NO,NO,NO,5.0,0.715566,0.850356
4687,4688,NO,NO,qgvZXHD9IvbkCB/tVbnbscJKQ5tquLjrODS1WZ3rHKo=,2aPjXEYHl/zwXs1o/1+8dOL/ep5424Ifn4Q3qavDG8Y=,1.304387,0.080829,0.950421,0.503305,0.582335,...,0.0,0.83,4676.0,3328.0,NO,NO,NO,0.0,0.267728,0.572070
4692,4693,NO,NO,cBZbp0dRvyJb1WGgcd+TnaipoQ0+rnfxOWhISfg0f18=,ZUWzVuCtD3tz2CjWMqvyxqIRZ2ZDZbRv5iwMw0y6XJY=,1.386755,0.418204,0.752041,1.375265,0.978828,...,0.0,0.75,4676.0,3307.0,YES,NO,YES,18.0,0.522830,0.972840
4694,4695,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.73,4400.0,3400.0,YES,YES,YES,0.0,0.800588,0.144545


In [ ]:
missing_percent_training_data=train_df.isna().sum().sort_values(ascending=False)*100/len(train_df)
missing_percent_training_data

x10    13.899532
x14    13.899532
x2     13.899532
x3     13.899532
x4     13.899532
         ...    
x54     0.000000
x53     0.000000
x52     0.000000
x51     0.000000
id      0.000000
Length: 146, dtype: float64

In [ ]:
missing_percent_train_target_data=train_target_df.isna().sum().sort_values(ascending=False)*100/len(train_target_df)
missing_percent_train_target_data

id     0.0
y25    0.0
y19    0.0
y20    0.0
y21    0.0
y22    0.0
y23    0.0
y24    0.0
y26    0.0
y1     0.0
y27    0.0
y28    0.0
y29    0.0
y30    0.0
y31    0.0
y32    0.0
y18    0.0
y17    0.0
y16    0.0
y15    0.0
y14    0.0
y13    0.0
y12    0.0
y11    0.0
y10    0.0
y9     0.0
y8     0.0
y7     0.0
y6     0.0
y5     0.0
y4     0.0
y3     0.0
y2     0.0
y33    0.0
dtype: float64

In [ ]:
missing_percent_test_data=test_df.isna().sum().sort_values(ascending=False)*100/len(test_df)
missing_percent_test_data

x26     14.7
x12     14.7
x2      14.7
x3      14.7
x4      14.7
        ... 
x53      0.0
x52      0.0
x51      0.0
x50      0.0
x145     0.0
Length: 146, dtype: float64

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns=train_df.select_dtypes(include=numerics).columns.to_list()
len(numeric_columns)

86

In [ ]:
object_columns=train_df.select_dtypes(include=object).columns
len(object_columns)

60

In [ ]:
train_df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [ ]:
train_df[object_columns]=train_df[object_columns].astype(str)
test_df[object_columns]=test_df[object_columns].astype(str)

In [ ]:
train_df[object_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4698 entries, 0 to 4697
Data columns (total 60 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   x1      4698 non-null   object
 1   x2      4698 non-null   object
 2   x3      4698 non-null   object
 3   x4      4698 non-null   object
 4   x10     4698 non-null   object
 5   x11     4698 non-null   object
 6   x12     4698 non-null   object
 7   x13     4698 non-null   object
 8   x14     4698 non-null   object
 9   x24     4698 non-null   object
 10  x25     4698 non-null   object
 11  x26     4698 non-null   object
 12  x30     4698 non-null   object
 13  x31     4698 non-null   object
 14  x32     4698 non-null   object
 15  x33     4698 non-null   object
 16  x34     4698 non-null   object
 17  x35     4698 non-null   object
 18  x41     4698 non-null   object
 19  x42     4698 non-null   object
 20  x43     4698 non-null   object
 21  x44     4698 non-null   object
 22  x45     4698 non-null   

In [ ]:
boolean_cols = []
alphanumeric_cols = []

for col in object_columns:
    if train_df[col][0].isupper()==True:
        boolean_cols.append(col)
    else:
        alphanumeric_cols.append(col)

In [ ]:
alphanumeric_cols

['x3', 'x4', 'x34', 'x35', 'x61', 'x64', 'x65', 'x91', 'x94', 'x95']

In [ ]:
train_df['x3'].value_counts()

nan                                             653
MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=    138
X/hdUOVR5KuExVGLzjhLcM2CyIqym9t0Nh+ZX05M+1w=     49
zq3MyXbqpnH+txFzhEWomNfZyeLXubUbZwdaRvKXLs4=     34
8Whd23AFTt1KV61HEnaVzYZCSZsw5sqqmf4WUmWd3bQ=     33
                                               ... 
NYgC4B7/9v8pYfB57/2cPT07vIzsWjfYX64yE7z8BwI=      1
OaZMZ63NJIdQsH4agfpFZwBhKGtiqC6iOejZ2wkhbOo=      1
6egBhJOGcCIbJKRQ7jx9E+MtCwsoNPrO3yod7kzScRY=      1
ja/F4vkTkStgwnODBOtB4S9majf41ltMXcsYuXFUBzU=      1
G8n9lQapNblKb05Up+TzwlMmPH6aUxH++7R6bI/4FuI=      1
Name: x3, Length: 2600, dtype: int64

In [ ]:
train_df[boolean_cols]=train_df[boolean_cols].astype('category')
test_df[boolean_cols]=test_df[boolean_cols].astype('category')

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4698 entries, 0 to 4697
Columns: 146 entries, id to x145
dtypes: category(50), float64(67), int64(19), object(10)
memory usage: 3.7+ MB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 146 entries, id to x145
dtypes: category(50), float64(55), int64(31), object(10)
memory usage: 1.6+ MB


In [ ]:
num_imputer=SimpleImputer(strategy='mean')

In [ ]:
obj_imputer=SimpleImputer(strategy='most_frequent')

In [ ]:
num_imputer.fit(train_df[numeric_columns])
train_df[numeric_columns]=num_imputer.transform(train_df[numeric_columns])

In [ ]:
num_imputer.fit(test_df[numeric_columns])

SimpleImputer()

In [ ]:
test_df[numeric_columns]=num_imputer.transform(test_df[numeric_columns])

In [ ]:
train_df[numeric_columns].isna().sum().sort_values(ascending=False)

id      0
x98     0
x111    0
x110    0
x109    0
       ..
x50     0
x49     0
x48     0
x47     0
x145    0
Length: 86, dtype: int64

In [ ]:
test_df[numeric_columns].isna().sum().sort_values(ascending=False)

id      0
x98     0
x111    0
x110    0
x109    0
       ..
x50     0
x49     0
x48     0
x47     0
x145    0
Length: 86, dtype: int64

In [ ]:
train_df.replace('nan', None, inplace=True)
test_df.replace('nan', None, inplace=True)

In [ ]:
train_df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1.0,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,...,0.0,0.810,3306.0,4676.0,YES,NO,YES,2.0,0.375535,0.464610
1,2.0,NaN,NaN,None,None,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.510,4678.0,3306.0,YES,NO,YES,4.0,0.741682,0.593630
2,3.0,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,...,0.0,0.850,4678.0,3306.0,NO,NO,NO,1.0,0.776467,0.493159
3,4.0,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.041471,0.940787,0.090851,0.556564,...,0.0,0.945,3306.0,4678.0,NO,NO,YES,3.0,0.168234,0.546582
4,5.0,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.415919,0.000000,1.000000,0.000000,0.375297,...,0.0,1.000,1263.0,892.0,NO,NO,NO,1.0,0.246637,0.361045


In [ ]:
for col in boolean_cols:
    train_df[col].replace({'YES':1, 'NO':0},inplace=True)
    test_df[col].replace({'YES':1, 'NO':0},inplace=True)

In [ ]:
train_df[boolean_cols]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,1,0,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4693,1,0,1,0,1,0,1,1,0,1,...,0,1,1,0,0,0,0,1,0,1
4694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,1,0,0,0,0,1,1,1
4695,0,0,0,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,1,0,1
4696,0,0,1,0,0,0,0,1,1,1,...,0,1,1,0,1,0,0,0,0,0


In [ ]:
obj_imputer.fit(train_df[boolean_cols])
train_df[boolean_cols]=obj_imputer.transform(train_df[boolean_cols])

In [ ]:
obj_imputer.fit(test_df[boolean_cols])
test_df[boolean_cols]=obj_imputer.transform(test_df[boolean_cols])

In [ ]:
test_df[boolean_cols]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,1,1,1,0,0,0,...,0,0,0,0,1,1,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,1,1,0,1
1996,0,0,1,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,1
1997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,0,0,0,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,0


In [ ]:
train_df[boolean_cols]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4693,1,0,1,0,1,0,1,1,0,1,...,0,1,1,0,0,0,0,1,0,1
4694,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,1
4695,0,0,0,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,1,0,1
4696,0,0,1,0,0,0,0,1,1,1,...,0,1,1,0,1,0,0,0,0,0


In [ ]:
train_df[boolean_cols].isna().sum().sort_values(ascending=False)

x1      0
x104    0
x74     0
x75     0
x85     0
x86     0
x87     0
x92     0
x93     0
x101    0
x102    0
x103    0
x105    0
x2      0
x115    0
x116    0
x117    0
x126    0
x127    0
x128    0
x129    0
x130    0
x140    0
x141    0
x73     0
x72     0
x71     0
x63     0
x10     0
x11     0
x12     0
x13     0
x14     0
x24     0
x25     0
x26     0
x30     0
x31     0
x32     0
x33     0
x41     0
x42     0
x43     0
x44     0
x45     0
x55     0
x56     0
x57     0
x62     0
x142    0
dtype: int64

In [ ]:
train_df[boolean_cols]=train_df[boolean_cols].astype('int')
test_df[boolean_cols]=test_df[boolean_cols].astype('int')

In [ ]:
train_df[numeric_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4698 entries, 0 to 4697
Data columns (total 86 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      4698 non-null   float64
 1   x5      4698 non-null   float64
 2   x6      4698 non-null   float64
 3   x7      4698 non-null   float64
 4   x8      4698 non-null   float64
 5   x9      4698 non-null   float64
 6   x15     4698 non-null   float64
 7   x16     4698 non-null   float64
 8   x17     4698 non-null   float64
 9   x18     4698 non-null   float64
 10  x19     4698 non-null   float64
 11  x20     4698 non-null   float64
 12  x21     4698 non-null   float64
 13  x22     4698 non-null   float64
 14  x23     4698 non-null   float64
 15  x27     4698 non-null   float64
 16  x28     4698 non-null   float64
 17  x29     4698 non-null   float64
 18  x36     4698 non-null   float64
 19  x37     4698 non-null   float64
 20  x38     4698 non-null   float64
 21  x39     4698 non-null   float64
 22  

In [ ]:
exclude_column = 'id'
numeric_columns = list(filter(lambda col: col != exclude_column, numeric_columns))

In [ ]:
X_train = train_df[numeric_columns+boolean_cols]
X_test  = test_df[numeric_columns+boolean_cols]
Y_train = train_target_df.loc[:9998,'y1':]

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(4698, 135)
(9999, 33)
(2000, 135)


<b>Training the Model & Predicting the Classes</b>

In [ ]:
X_train

,x5,x6,x7,x8,x9,x15,x16,x17,x18,x19,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0.576561,0.073139,0.481394,0.115697,0.472474,42.0,0.396065,3.0,6.0,0.991018,...,0,1,1,0,0,0,0,1,0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,1,0,1
2,1.341803,0.051422,0.935572,0.041440,0.501710,2.0,0.838475,3.0,5.0,0.966122,...,0,0,0,0,1,0,0,0,0,0
3,0.653912,0.041471,0.940787,0.090851,0.556564,37.0,0.127405,8.0,15.0,0.959171,...,0,1,1,0,0,0,0,0,0,1
4,1.415919,0.000000,1.000000,0.000000,0.375297,1.0,0.523543,4.0,11.0,0.963004,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4693,0.498503,0.065868,0.243798,0.124679,0.630974,13.0,0.181352,1.0,9.0,0.997434,...,0,1,1,0,0,0,0,1,0,1
4694,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0,1,1,0,0,0,0,1,1,1
4695,1.415919,0.000000,1.000000,0.000000,0.232779,6.0,0.547085,6.0,8.0,0.939462,...,0,1,1,0,1,0,1,1,0,1
4696,1.415919,0.000000,1.000000,0.000000,0.169438,2.0,0.635650,10.0,9.0,0.915919,...,0,1,1,0,1,0,0,0,0,0


In [ ]:
def hash_columns(df, columns_to_hash):
    hashed_df = pd.DataFrame()

    for column in alphanumeric_cols:
        hashed_df['hashed_'+column] = df[column].apply(hash)

    return hashed_df

hashed_train_df = hash_columns(train_df,alphanumeric_cols)

hashed_test_df = hash_columns(test_df,alphanumeric_cols)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_hashed_train_df = pd.DataFrame(scaler.fit_transform(hashed_train_df), columns=hashed_train_df.columns)
scaled_hashed_test_df = pd.DataFrame(scaler.transform(hashed_test_df), columns=hashed_test_df.columns)

In [ ]:
scaled_hashed_train_df

,hashed_x3,hashed_x4,hashed_x34,hashed_x35,hashed_x61,hashed_x64,hashed_x65,hashed_x91,hashed_x94,hashed_x95
0,0.919930,0.630119,0.051148,0.556771,0.481370,0.736361,0.154603,0.007668,0.121028,0.787327
1,0.500132,0.499933,0.892283,0.540694,0.518328,0.374623,0.446080,0.345971,0.960831,0.456550
2,0.555071,0.456677,0.616415,0.980583,0.259192,0.024944,0.610867,0.456141,0.960083,0.258575
3,0.370382,0.934410,0.370214,0.934238,0.421698,0.424600,0.934238,0.920586,0.424390,0.934151
4,0.466730,0.301749,0.975882,0.239098,0.581938,0.841671,0.035180,0.426289,0.411686,0.606691
...,...,...,...,...,...,...,...,...,...,...
4693,0.917578,0.130128,0.153425,0.641555,0.060958,0.008029,0.641555,0.349167,0.124603,0.986268
4694,0.500132,0.499933,0.221311,0.907460,0.635398,0.948115,0.112372,0.977381,0.055572,0.538440
4695,0.264833,0.203958,0.932758,0.690663,0.972659,0.933092,0.690663,0.526890,0.643400,0.669327
4696,0.912778,0.773642,0.166474,0.782110,0.605054,0.166853,0.782110,0.439212,0.645036,0.532427


In [ ]:
X_train_hash = pd.concat([X_train,scaled_hashed_train_df], axis=1)
X_test_hash = pd.concat([X_test,scaled_hashed_test_df], axis=1)

In [ ]:
print(X_train_hash.shape)
print(Y_train.shape)
print(X_test_hash.shape)

(4698, 145)
(9999, 33)
(2000, 145)


In [ ]:
print(X_train_hash.shape)
print(Y_train.shape)

(4698, 145)
(9999, 33)


In [ ]:
print(X_train_hash.isnull().sum().sum())
print(Y_train.isnull().sum().sum())

0
0


In [ ]:
print(X_train_hash.duplicated().sum())
print(Y_train.duplicated().sum())

23
9942


In [ ]:
print(X_train_hash.dtypes)
print(Y_train.dtypes)

x5            float64
x6            float64
x7            float64
x8            float64
x9            float64
               ...   
hashed_x64    float64
hashed_x65    float64
hashed_x91    float64
hashed_x94    float64
hashed_x95    float64
Length: 145, dtype: object
y1     int64
y2     int64
y3     int64
y4     int64
y5     int64
y6     int64
y7     int64
y8     int64
y9     int64
y10    int64
y11    int64
y12    int64
y13    int64
y14    int64
y15    int64
y16    int64
y17    int64
y18    int64
y19    int64
y20    int64
y21    int64
y22    int64
y23    int64
y24    int64
y25    int64
y26    int64
y27    int64
y28    int64
y29    int64
y30    int64
y31    int64
y32    int64
y33    int64
dtype: object


In [ ]:
Y_train = Y_train[:4698]

In [ ]:
X_train_hash = X_train_hash.append(pd.DataFrame(np.zeros((9999 - 4698, 145)), columns=X_train_hash.columns))

In [ ]:
model = OneVsRestClassifier(RandomForestClassifier())
model.fit(X_train_hash,Y_train)
Y_pred_hash = model.predict(X_train_hash)
ham_loss = hamming_loss(Y_train, Y_pred_hash)
print("Hamming Loss for Training Data",ham_loss)

In [ ]:
print("Accuracy Score for Training Data",accuracy_score(Y_train,Y_pred_hash))

In [ ]:
classification_rep = classification_report(Y_train,Y_pred_hash)

print('Classification Report:\n', classification_rep)

In [ ]:
test_df.id

In [ ]:
target=[]
for num in test_df.id:
    for i in range(1,34):
        target.append(str(num)+'_y'+str(i))

In [ ]:
len(target)

In [ ]:
submission_df=pd.DataFrame(target,columns=['id'])

In [ ]:
submission_df

In [ ]:

# prediction for Test data
predictions_hash = model.predict(X_test_hash)

In [ ]:
prediction_hash_list=[]
for i in range(len(predictions_hash)):
    prediction_hash_list.extend(predictions_hash.tolist()[i])

In [ ]:
pred_hash_df=pd.DataFrame(prediction_hash_list,columns=['pred'])

In [ ]:
test_prediction_hash_df=pd.concat([submission_df,pred_hash_df],axis=1)

In [ ]:
test_prediction_hash_df.to_csv('test_prediction_final.csv',index=None)